In [33]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Cleaning

In [50]:
import pandas as pd
import seaborn as sns

In [51]:
df = pd.read_csv('../train.csv')

In [52]:
df.shape

(250000, 32)

In [53]:
df[df.Prediction=='s'].shape

(85667, 32)

In [54]:
df[df.Prediction=='b'].shape

(164333, 32)

In [55]:
feat_names = [col for col in df.columns if col not in ['Id','Prediction']]

for colname in feat_names:
    df.loc[df[colname]==-999.000,[colname]]=df[colname][df[colname]!=-999.000].median()

In [56]:
df.describe()

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,224999.500000,120.417434,49.239819,81.181982,57.895962,2.193104,268.220619,-0.411629,2.373100,18.917332,...,-0.010119,209.797178,0.979176,77.124366,-0.001966,-0.020629,50.739149,-0.010535,-0.001879,73.064591
std,72168.927986,52.859218,35.344886,40.828691,63.655682,0.948104,224.229668,1.948520,0.782911,22.273494,...,1.812223,126.499506,0.977426,47.939899,1.382703,1.405084,17.792135,1.094446,0.978743,98.015662
min,100000.000000,9.044000,0.000000,6.329000,0.000000,0.000000,13.602000,-18.066000,0.208000,0.000000,...,-3.142000,13.678000,0.000000,30.000000,-4.499000,-3.142000,30.000000,-4.500000,-3.142000,0.000000
25%,162499.750000,95.665000,19.241000,59.388750,14.068750,2.107000,225.885000,-0.244000,1.810000,2.841000,...,-1.575000,123.017500,0.000000,57.439000,-0.433000,-0.556000,47.902000,-0.010000,-0.002000,0.000000
50%,224999.500000,112.406000,46.524000,73.752000,38.467500,2.107000,225.885000,-0.244000,2.491500,12.315500,...,-0.024000,179.739000,1.000000,65.561000,0.000000,-0.033000,47.902000,-0.010000,-0.002000,40.512500
75%,287499.250000,130.606250,73.598000,92.259000,79.169000,2.107000,225.885000,-0.244000,2.961000,27.591000,...,1.561000,263.379250,2.000000,75.349000,0.433000,0.503000,47.902000,-0.010000,-0.002000,109.933750
max,349999.000000,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,...,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


In [57]:
df.shape

(250000, 32)

In [58]:
sum(df["DER_mass_MMC"]==-999.00)

0

In [59]:
df.to_csv('../train_mod.csv', index=False)

In [59]:
phys_feats = ['Id','Prediction'] + DER_features

In [60]:
df_phys = df[phys_feats]

In [62]:
df_phys.to_csv('../train_phys.csv', index=False)

In [66]:
df[DER_features].as_matrix().shape

(250000, 13)

In [78]:
test = pd.read_csv('../test.csv')
test_phys = test[phys_feats]

In [82]:
test_phys.to_csv('../test_phys.csv',index=False)

## PCA train

In [11]:
M = df[feat_names].as_matrix()
U, s, Vt = np.linalg.svd(M, full_matrices=False)
V = Vt.T

# PCs are already sorted by descending order 
# of the singular values (i.e. by the
# proportion of total variance they explain)

# if we use all of the PCs we can reconstruct the noisy signal perfectly
S = np.diag(s)
Mhat = np.dot(U, np.dot(S, V.T))
print("Using all PCs, MSE = %.6G" %(np.mean((M - Mhat)**2)))

# if we use only the first 20 PCs the reconstruction is less accurate
Mhat2 = np.dot(U[:, :20], np.dot(S[:20, :20], V[:,:20].T))
print("Using first 20 PCs, MSE = %.6G" %(np.mean((M - Mhat2)**2)))

Using all PCs, MSE = 3.52782E-25
Using first 20 PCs, MSE = 0.150423


In [12]:
singulars = np.diag(S)**2
singulars.cumsum()/singulars.sum()

array([ 0.8231826 ,  0.93230183,  0.97205019,  0.97924848,  0.98508109,
        0.98984158,  0.99247884,  0.99494948,  0.99635745,  0.99760283,
        0.99863844,  0.99941667,  0.99992348,  0.99993718,  0.99994968,
        0.99995916,  0.99996818,  0.99997438,  0.99997951,  0.99998454,
        0.99998818,  0.99999129,  0.9999941 ,  0.99999642,  0.99999859,
        0.99999922,  0.99999964,  0.99999991,  1.        ,  1.        ])

In [13]:
up_to = 20
Mhat = np.dot(U[:, :up_to], np.sqrt(S[:up_to, :up_to]))
Mhat.shape

(250000, 20)

In [14]:
PCS_train = np.dot(np.sqrt(S[:up_to, :up_to]), V[:,:up_to].T).T
PCS_train.shape

(30, 20)

In [15]:
df.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,...,-0.277,258.733,2.0,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,2.107,225.885,-0.244,3.473,...,-1.916,164.546,1.0,46.226,0.725,1.158,47.902,-0.01,-0.002,46.226
2,100002,b,112.406,162.172,125.953,35.635,2.107,225.885,-0.244,3.148,...,-2.186,260.414,1.0,44.251,2.053,-2.028,47.902,-0.01,-0.002,44.251
3,100003,b,143.905,81.417,80.943,0.414,2.107,225.885,-0.244,3.310,...,0.060,86.062,0.0,65.561,0.000,-0.033,47.902,-0.01,-0.002,0.000
4,100004,b,175.864,16.915,134.805,16.405,2.107,225.885,-0.244,3.891,...,-0.871,53.131,0.0,65.561,0.000,-0.033,47.902,-0.01,-0.002,0.000


In [16]:
##Dummy names for PCA features (must match format)
df_pca = pd.DataFrame(Mhat)
df_pca['Id'] = df['Id']
df_pca['Prediction'] = df['Prediction']
df_pca.columns = feat_names[:up_to] + ['Id','Prediction']
df_pca = df_pca[['Id','Prediction']+feat_names[:up_to]]
df_pca.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met
0,100000,s,-0.769838,-0.543040,-0.254366,0.122660,-0.292200,-0.290423,-0.066440,0.045273,...,-0.031438,0.159607,0.044618,0.065721,0.017606,0.034698,0.110437,-0.005449,-0.002059,-0.083173
1,100001,b,-0.746935,-0.021490,-0.450921,-0.120646,0.002360,0.013581,-0.171292,-0.011218,...,0.014835,-0.079058,-0.145778,-0.056469,0.099484,0.038748,0.066410,0.007465,-0.009017,0.000586
2,100002,b,-0.904274,-0.252481,-0.490765,0.280186,0.386489,-0.602813,-0.082314,-0.715058,...,0.130950,-0.075938,0.228257,-0.069395,0.045872,-0.033533,0.001600,-0.082832,-0.092660,0.018978
3,100003,b,-0.594008,0.223270,-0.535427,-0.165784,0.165640,-0.199944,0.107131,0.127347,...,-0.139498,0.022658,0.014392,0.074639,0.015575,0.059463,-0.043838,-0.026064,-0.040767,0.008549
4,100004,b,-0.566397,0.275458,-0.636252,-0.407153,-0.286477,0.171639,0.156505,0.230334,...,0.095634,-0.009633,-0.222229,-0.079581,0.040057,0.000899,-0.027718,-0.015715,-0.011459,-0.000286


In [17]:
df_pca.shape

(250000, 22)

In [18]:
df_pca.to_csv('../train_pca.csv',index=False)

## PCA test

In [60]:
df_pca_test = pd.read_csv('../test.csv')
df_pca_test.shape

(568238, 32)

In [61]:
feat_names = [col for col in df_pca_test.columns if col not in ['Id','Prediction']]

In [21]:
pca_test = df_pca_test[feat_names].as_matrix()
pca_test.shape

(568238, 30)

In [22]:
pca_test = np.dot(pca_test,PCS_train)
pca_test.shape

(568238, 20)

In [23]:
pca_test = pd.DataFrame(pca_test)
pca_test['Id'] = df_pca_test['Id']
pca_test['Prediction'] = df_pca_test['Prediction']
pca_test.columns = feat_names[:up_to] + ['Id','Prediction']
pca_test = pca_test[['Id','Prediction'] + feat_names[:up_to]]
pca_test.shape

(568238, 22)

In [24]:
pca_test.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met
0,350000,?,-137351.948492,22192.003098,-18029.576894,984.706527,7691.652369,-2615.330753,432.169987,4059.423219,...,-2752.418848,292.968795,311.324087,19.887650,-36.920995,65.359929,-17.140139,-8.339968,-5.822458,1.617966
1,350001,?,-179867.751988,-2331.484286,-14238.875034,990.215239,700.027427,-1889.995886,102.242738,-3109.724679,...,62.932828,-853.152559,-1014.153567,22.453747,-26.012374,-103.568756,-40.421765,-2.865803,11.091086,-3.942387
2,350002,?,-153518.811579,14658.983079,-24360.761221,-1357.676677,290.089814,-2164.740222,4242.702420,-640.600716,...,71.616095,-258.511736,415.152368,62.245676,10.791008,-85.966516,-12.843907,-5.712181,12.589230,-2.301353
3,350003,?,-166742.950919,7399.379408,-25143.885374,3492.205755,-2158.616233,2161.549444,3386.814426,1235.485043,...,2.391499,-286.307508,956.057499,-80.504179,-26.207603,8.967708,1.619740,1.319876,14.007009,-3.670213
4,350004,?,-632437.284863,-141068.749431,78245.618974,18715.424491,-16951.449081,-32743.328433,3836.826536,7392.087744,...,-524.609267,-973.260609,-990.801357,-25.417641,-30.599906,-78.944951,-75.417246,-3.814281,15.366144,33.129219


In [25]:
pca_test.to_csv('../test_pca.csv',index=False)